In [1]:
import os
import sys
import pandas as pd

sys.path.append("../")
from utils.io import read_parameters
from src.plot_trajectories import plot_trajectories_per_file
from src.plot_trajectories import plot_trajectories_from_origin_per_file
from src.plot_trajectories import plot_trajectories_from_origin_per_condition

In [2]:
# read parameters and key file

parameter_file  = "../local/parameters.yml"
parameters = read_parameters(parameter_file)       


key_file_path = parameters["key_file"]
key_file = pd.read_csv(key_file_path)
print(key_file.head())

             experimentID                                  filename  color  \
0  230901_siCTRL_1_mosaic  /siCTRL_mosaic/230901_siCTRL_green_1.csv  green   
1  230901_siCTRL_1_mosaic    /siCTRL_mosaic/230901_siCTRL_red_1.csv    red   
2  230901_siCTRL_2_mosaic  /siCTRL_mosaic/230901_siCTRL_green_2.csv  green   
3  230901_siCTRL_2_mosaic    /siCTRL_mosaic/230901_siCTRL_red_2.csv    red   
4    230915_siCTRL_mosaic    /siCTRL_mosaic/230915_siCTRL_green.csv  green   

   mosaic treatment             condition    date  
0       1    siCTRL  mosaic_siCTRL_siCTRL  230901  
1       1    siCTRL  mosaic_siCTRL_siCTRL  230901  
2       1    siCTRL  mosaic_siCTRL_siCTRL  230901  
3       1    siCTRL  mosaic_siCTRL_siCTRL  230901  
4       1    siCTRL  mosaic_siCTRL_siCTRL  230915  


In [3]:
# create output folder

output_folder = parameters["output_folder"]
print("Output folder:", output_folder)

subfolder = "trajectory_plots"
if not os.path.exists(output_folder + "/" + subfolder):
    os.mkdir(output_folder + "/" + subfolder)


Output folder: /media/fjodor/TOSHIBA/Dropbox/MDC_Projects/Olya_migration/mosaic/output_240605/


In [4]:
plot_trajectories_from_origin_per_condition(parameters, key_file, subfolder = "tracking_data", number_of_tracks_per_condition = parameters["number_of_cells_per_condition"])

Plotting trajectories(starting at origin) for condition:  mosaic_siCTRL_siCTRL
Plotting trajectories (starting at origin) for condition mosaic_siCTRL_siCTRL and siCTRL: 
Number of files: 10 
sample 30 tracks per file
Available tracks: 1590
sampled 30 tracks for file tracking_data_siCTRL_green_230901_siCTRL_1_mosaic.csv
Available tracks: 3464
sampled 30 tracks for file tracking_data_siCTRL_red_230901_siCTRL_1_mosaic.csv
Available tracks: 1755
sampled 30 tracks for file tracking_data_siCTRL_green_230901_siCTRL_2_mosaic.csv
Available tracks: 3386
sampled 30 tracks for file tracking_data_siCTRL_red_230901_siCTRL_2_mosaic.csv
Available tracks: 1042
sampled 30 tracks for file tracking_data_siCTRL_green_230915_siCTRL_mosaic.csv
Available tracks: 3746
sampled 30 tracks for file tracking_data_siCTRL_red_230915_siCTRL_mosaic.csv
Available tracks: 1681
sampled 30 tracks for file tracking_data_siCTRL_green_230922_siCTRL_1_mosaic.csv
Available tracks: 4814
sampled 30 tracks for file tracking_data_s